In [6]:
import os
import time
import numpy as np
import pandas as pd

# Read book value data    
book_df = pd.read_excel('D:/Thesis_data/Book.xlsx')
a = []
for _, row in book_df.iterrows():
    if not pd.isna(row['Stockholders Equity - Parent']):
        row['book value'] = row['Stockholders Equity - Parent']
    elif (not pd.isna(row['Common/Ordinary Equity - Total'])) and (not pd.isna(row['Preferred/Preference Stock (Capital) - Total'])): 
        row['book value'] = row['Common/Ordinary Equity - Total'] + row['Preferred/Preference Stock (Capital) - Total']
    elif (not pd.isna(row['Assets - Total'])) and (not pd.isna(row['Liabilities - Total'])) and (not pd.isna(row['Noncontrolling Interest (Balance Sheet)'])):
        if not pd.isna(row['Preferred/Preference Stock (Capital) - Total']):
            row['book value'] = row['Assets - Total'] - row['Liabilities - Total'] - row['Noncontrolling Interest (Balance Sheet)'] - row['Preferred/Preference Stock (Capital) - Total']
        elif not pd.isna(row['Preferred Stock - Liquidating Value']):
            row['book value'] = row['Assets - Total'] - row['Liabilities - Total'] - row['Noncontrolling Interest (Balance Sheet)'] - row['Preferred Stock - Liquidating Value']
        elif not pd.isna(row['Preferred Stock - Redemption Value']):
            row['book value'] = row['Assets - Total'] - row['Liabilities - Total'] - row['Noncontrolling Interest (Balance Sheet)'] - row['Preferred Stock - Redemption Value']
    try:
        row['B'] = row['book value'] / row['Common Shares Outstanding']
    except:
        pass
    a.append(row)
b = pd.DataFrame(a)
book = b[['Data Date','Data Year - Fiscal','Ticker Symbol','B']]
book.columns = ['Data Date', 'Year', 'Ticker', 'B']
book = book[book['Year'].notnull()]
book['Year'] = book['Year'].map(lambda x: str(int(x)))

# Read price
price_df = pd.read_excel('D:/Thesis_data/market.xlsx')
price_df['Price or Bid/Ask Average'] = price_df['Price or Bid/Ask Average'].map(abs)
price_df['market size'] = price_df['Price or Bid/Ask Average'] * price_df['Shares Outstanding']
price_df['year'] = price_df['Names Date'].map(lambda x: str(x)[:4])
price_df['month'] = price_df['Names Date'].map(lambda x: str(x)[4:6])
price = price_df[['Ticker Symbol','market size', 'Returns', 'Price or Bid/Ask Average', 'year', 'month']]
price.columns = ['Ticker', 'market size', 'R', 'price', 'year', 'month']

# build book dict to find Book value
book_dict = {}
for ticker in book['Ticker'].unique():
    book_dict[ticker] = {}
for _, row in book.iterrows():
    book_dict[row['Ticker']][row['Year']] = row['B']
df = price
df['B'] = 0
final_rows=[]
for i, row in df.iterrows():
    if int(row['month']) < 7:
        fiscal_year_to_see = str(int(row['year']) - 2)
    else:
        fiscal_year_to_see = str(int(row['year']) - 1)
    try:
        row['B'] = book_dict[row['Ticker']][fiscal_year_to_see]
    except:
        row['B'] = np.nan
    final_rows.append(row)
df_final = pd.DataFrame(final_rows)
df_final.dropna().to_excel('D:/Thesis_data/factorData.xlsx')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


To compute total book value of equity (BE), we prefer stockholders’ equity (SEQ). 
If that is unavailable,we use the sum of common equity (CEQ) and preferredstock (PSTK). 
If both SEQ and CEQ are unavailable, we proxy book equity by total assets (AT), minus the sum of
total liability (LT), minority interest (MIB), and preferred stocks (PSTKRV, PSTKL, or PSTK, depending on availability).

To compute book value per share (B), we divide by common shares outstanding (CSHPRI). 
If CSHPRI is missing, we use compute company-level total shares outstanding by summing issue-level shares (CSHOI) at fiscal year-end for securities with an earnings participation flag in the security-pricing file.

In [ ]:
'''
|------------------------------------------------------------------------------|
| Ticker |  date  | Book to market | Market size | Book value | Price | Return |
|------------------------------------------------------------------------------|
|  AAPL  | 200505 |                |    200505   |   200312   | 200505| 200505 |                 
|------------------------------------------------------------------------------|
|  AAPL  | 200506 |                |    200506   |   200412   | 200506| 200506 |      
|------------------------------------------------------------------------------|
|  AAPL  | 200507 |                |    200507   |   200412   | 200507| 200507 |                
|------------------------------------------------------------------------------|

'''

In [3]:
price_df[price_df['Price or Bid/Ask Average'] < 0 ]

,PERMNO,Names Date,Standard Industrial Classification Code,Ticker Symbol,Price or Bid/Ask Average,Returns,Shares Outstanding,market size,year,month
31,10001,20070831,4920,EWST,-14.5400,0.027562,2859.0,-41569.8600,2007,08
574,10028,20050429,5944,DGSE,-2.3550,-0.094231,4913.0,-11570.1150,2005,04
576,10028,20050630,5944,DGSE,-2.6500,0.095041,4913.0,-13019.4500,2005,06
577,10028,20050729,5944,DGSE,-2.4450,-0.077358,4913.0,-12012.2850,2005,07
578,10028,20050831,5944,DGSE,-2.5350,0.03681,4913.0,-12454.4550,2005,08
584,10028,20060228,5944,DGSE,-1.9950,-0.0025,4913.0,-9801.4350,2006,02
619,10028,20090130,5094,DGC,-1.3750,-0.191176,9834.0,-13521.7500,2009,01
707,10028,20160531,5094,DGSE,-0.5300,-0.032317,12329.0,-6534.3700,2016,05
1293,10056,20051230,6036,NMIL,-30.3700,-0.000987,4086.0,-124091.8200,2005,12
1301,10056,20060831,6036,NMIL,-40.4050,0.010125,4075.0,-164650.3750,2006,08


In [4]:
import math
math.abs(-8)

AttributeError: module 'math' has no attribute 'abs'

In [5]:
abs(-8)

8